In [3]:
import os
os.getcwd()

import numpy as np
from math import sqrt
mrt = np.genfromtxt("MRT Coordinates in pixels.csv", delimiter=","
                ,skip_header=1,
                dtype=[('Map_Code','U5'),('Town','U20'),('U/C','i1'),('MRT','U20'),
                      ('Pixels Scale per 500m','f8'),('MRT X Coordinates','f8'),
                      ('MRT Y Coordinates','f8')],
                   missing_values=['na','-',''],filling_values=0)
#print(mrt[mrt['Map_Code']=='amk'])

townmapRaw = np.genfromtxt("Townmap Raw and Parsed from HTML/townmap_xy.csv", delimiter=",",
                          skip_header=1,
                          dtype=[('Town_Code','U8'),('Map_Code','U8'),('NG','U8'),('BLK','U8'),('X','i8'),('Y','i8')]
                          , missing_values=['na','-',''],filling_values=0)
#print(townmapRaw)
overallToSaveLines = []
#if townmapRaw['Map_Code']=='tp2'and mrt['Map_Code']==townmapRaw['Map_Code']:
#    for i in len(townmapRaw['Map_Code']=='tp2'):
#        ShortestDist = min(sqrt())
for j in townmapRaw:
    # j : townCode, mapCode, ng, Blks, x ,y
    mapCode = j[1].strip() #get rid of extra spaces from and back
    #print(mapCode)
    x_blk = j[4]
    y_blk = j[5]
    blk_BLK= j[3].strip()
    #print(j)
    #print(mrt[mrt['Map_Code'] == mapCode])
    
    # Some arbitrary number since some of the townmaps(png) are missing but got values for townmap_raw
    distlist= [99999999]  # since some towns do not have 4 or 5 room flats
    minDistSoFar = 99999999
    closestMrtSoFar = ""
    
    for i in mrt[mrt['Map_Code'] == mapCode]:
        mrtStation = i[3]
        x_mrt = i[5]
        y_mrt = i[6]
        scale_mrt = i[4] 
        town = i[1]
        
        # Finding distance between MRT and BLKS within each Townmap in pixel units
        dist = sqrt((x_blk-x_mrt)**2 + (y_blk-y_mrt)**2)        
        # Normalising distance per Townmap to metres
        dist = (dist/scale_mrt)* 500  #data was in pixels per 500m
        distlist.append(dist)
        if dist < minDistSoFar:
            minDistSoFar = dist #same as shortestDist
            closestMrtSoFar = mrtStation
        # find the shortest distance per Townmap
        #print(x_blk,y_blk,blk_BLK,x_mrt,y_mrt,scale_mrt,mrtStation,dist)

    # in j loop, only single values of these
    shortestDist = min(distlist)
    newJ = list(j)
    newJ.append(closestMrtSoFar)
    newJ.append(shortestDist)
    newJ.append(town)
    
    
    #print(j)
    #print(newJ)
    #toPrint = ",".join(newJ)
    # to put a comma in between all the columns when printed out
    toPrint = ""
    for field in newJ:
        toPrint = "%s%s," % (toPrint , field.__str__().strip()) #str(field).strip = field.__str__().strip
    #print(toPrint)
    overallToSaveLines.append(toPrint)
    #print(newJ,closestMrtSoFar,blk_BLK , shortestDist , minDistSoFar )
#print(overallToSaveLines)


## save to file
with open('your_file.csv', 'w') as f:
    f.write("townCode,mapCode,ng,blk,blkX,blkY,closestMrtStation,distToMrt,town,dummy\n")
    for item in overallToSaveLines:
        f.write("%s\n" % item)
